In [165]:
%%time

import requests
from bs4 import BeautifulSoup
import pandas as pd

start_year = 2005
end_year = 2021

Years = range(start_year,end_year+1)

Countries = ['Portugal','Spain','England','Italy','Germany','France']

# creation of an empty data frame for all countries, all years
table_fifa_all_years = pd.DataFrame()

for Year in Years:
    
    year = str(Year)[-2:]

    # creation of an empty data frame for all countries, one year
    table_fifa_all_countries = pd.DataFrame()

    for Country in Countries:
        if Country == 'Portugal':
            league = 308
        elif Country == 'Spain':
            league = 53
        elif Country == 'England':
            league = 13
        elif Country == 'Italy':
            league = 31
        elif Country == 'Germany':
            league = 19
        elif Country == 'France':
            league = 16
        else:
            print('Country not available, please choose between Portugal,\
        Spain, England, Italy, Germany or France')

        
        
        # Per each country:
        webpage = pd.read_html(f'https://www.fifaindex.com/teams/fifa{year}/?league={league}&order=desc')
        table_fifa_country = pd.DataFrame(webpage[0])
        table_fifa_country.dropna(subset = ["Name"], inplace=True)
        table_fifa_country.drop(table_fifa_country.columns.difference(
            ['Name','ATT','MID','DEF','OVR']), 1, inplace=True)
        table_fifa_country.insert(0, "Year", Year, True)
        table_fifa_country.insert(1, "Country", Country, True)
        table_fifa_country.reset_index(drop=True,inplace=True)
        

        # Concat the table created for one team with the empty dataframe
        table_fifa_all_countries = pd.concat(
                [table_fifa_all_countries,
                 table_fifa_country],
            ignore_index = True, axis = 0)

        
    
    # Concat the table created for one year with the empty dataframe
    table_fifa_all_years = pd.concat(
            [table_fifa_all_years,
             table_fifa_all_countries],
         ignore_index = True, axis = 0)
    

table_fifa_all_years

CPU times: user 3.62 s, sys: 118 ms, total: 3.74 s
Wall time: 6.31 s


,Year,Country,Name,ATT,MID,DEF,OVR
0,2005,Portugal,FC Porto,85.0,82.0,77.0,81.0
1,2005,Portugal,Benfica,80.0,74.0,74.0,75.0
2,2005,Portugal,Sporting Lisbon,63.0,70.0,65.0,68.0
3,2005,Portugal,Belenenses,67.0,62.0,62.0,64.0
4,2005,Portugal,Nacional,69.0,59.0,61.0,63.0
...,...,...,...,...,...,...,...
1947,2021,France,FC Nantes,71.0,74.0,74.0,73.0
1948,2021,France,Nîmes Olympique,71.0,73.0,73.0,73.0
1949,2021,France,FC Metz,70.0,70.0,73.0,72.0
1950,2021,France,Dijon FCO,73.0,71.0,70.0,71.0


In [301]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


# Check if the number of teams per year and league of Fifa Index site
# matches the number of teams in the original table

info_all_missing_teams = []

for Year in Years:
    for Country in Countries:
        df_teams_fifa = table_fifa_all_years[((
        table_fifa_all_years['Year'] == int(f'{Year}')) &
        (table_fifa_all_years['Country'] == f'{Country}'
    ))]
        list_teams_fifa = list(set(list(df_teams_fifa['Name'])))
        len(list_teams_fifa)

        df_teams = table_all_years[(
        (table_all_years['Year'] == int(f'{Year}')) &
        (table_all_years['Country'] == f'{Country}') &
        (table_all_years['Competition'] == 'National League') 
    )]
        list_teams = list(set(list(df_teams['Home_team'])))
        len(list_teams)
        
        if (len(list_teams_fifa)-len(list_teams)) < 0:
            info_missing_teams = (Year,Country,f'Number of teams missing in FIFA table: {-(len(list_teams_fifa)-len(list_teams))}')
            info_all_missing_teams.append(list(info_missing_teams))
        elif (len(list_teams_fifa)-len(list_teams)) > 0:
            info_missing_teams = (Year,Country,f'Number of teams missing in original table: {len(list_teams_fifa)-len(list_teams)}')
            info_all_missing_teams.append(list(info_missing_teams))

# info_all_missing_teams are the years and countries with teams missing



for first_teams_missing in info_all_missing_teams:
    aux_table = table_all_years[(table_all_years['Year']==int(first_teams_missing[0])) & 
                                   (table_all_years['Country']==first_teams_missing[1]) & 
                                   (table_all_years['Competition']=='National League')
                               ]
    original_list = list(set(aux_table['Home_team']))
    
    print('Original table',first_teams_missing[0],first_teams_missing[1],',',
          len(original_list),'teams')
    print(original_list,'\n')
    
    
    aux_table = table_fifa_all_years[(table_fifa_all_years['Year']==int(first_teams_missing[0])) & 
                                   (table_fifa_all_years['Country']==first_teams_missing[1]) 
                               ]
    fifa_list = list(set(aux_table['Name']))
    
    print('Fifa table',first_teams_missing[0],first_teams_missing[1],',',
          len(fifa_list),'teams')
    print(fifa_list,'\n')
    





    d = {'Name_from_original': original_list}
    df1 = pd.DataFrame(data=d)

    f = {'Name_from_fifa': fifa_list}
    df2 = pd.DataFrame(data=f)

    df1['Name_from_fifa_suggested_by_fuzzywuzzy'] = df1['Name_from_original'].apply(
        (lambda x: process.extractOne(x, df2['Name_from_fifa'].to_list(),score_cutoff=50)[0])
    )

    df1['fuzzywuzzy_score'] = df1['Name_from_original'].apply(
        (lambda x: process.extractOne(x, df2['Name_from_fifa'].to_list(),score_cutoff=50)[1])
    )

    df1['Count_suggested_name'] = df1['Name_from_fifa_suggested_by_fuzzywuzzy'].apply(
        (lambda x: df1['Name_from_fifa_suggested_by_fuzzywuzzy'].to_list().count(x))
    )

    df1['Suggested_names_Correction'] = df1.apply(
        lambda x: x['Name_from_fifa_suggested_by_fuzzywuzzy'] 
        if x['Count_suggested_name'] == 1
        else (x['Name_from_fifa_suggested_by_fuzzywuzzy'] if 
        x['fuzzywuzzy_score'] == df1[df1['Name_from_fifa_suggested_by_fuzzywuzzy']==x['Name_from_fifa_suggested_by_fuzzywuzzy']]['fuzzywuzzy_score'].max()
        else 'NONE'), axis = 1
    )

    
    list_teams_missing = list(df1[df1['Suggested_names_Correction']=='NONE']['Name_from_original'])
    print(list_teams_missing,'\n\n\n')

    
    break
    
    
df1

Original table 2012 England , 20 teams
['Chelsea', 'Man. Utd', 'Arsenal', 'Liverpool', 'Sunderland', 'Newcastle', 'West Bromwich Albion', 'Norwich City', 'Queens Park Rangers', 'Swansea City', 'Wigan Athletic', 'Everton', 'Blackburn Rovers', 'Bolton Wanderers', 'Stoke City', 'Man. City', 'Fulham', 'Tottenham Hotspur', 'Aston Villa', 'Wolves'] 

Fifa table 2012 England , 19 teams
['Chelsea', 'Liverpool', 'Manchester City', 'Sunderland', 'West Bromwich Albion', 'Manchester United', 'Norwich City', 'Queens Park Rangers', 'Swansea City', 'Wigan Athletic', 'Everton', 'Newcastle United', 'Blackburn Rovers', 'Bolton Wanderers', 'Stoke City', 'Fulham', 'Tottenham Hotspur', 'Wolverhampton Wanderers', 'Aston Villa'] 

['Arsenal'] 





,Name_from_original,Name_from_fifa_suggested_by_fuzzywuzzy,fuzzywuzzy_score,Count_suggested_name,Suggested_names_Correction
0,Chelsea,Chelsea,100,1,Chelsea
1,Man. Utd,Manchester United,56,1,Manchester United
2,Arsenal,Swansea City,57,2,NONE
3,Liverpool,Liverpool,100,1,Liverpool
4,Sunderland,Sunderland,100,1,Sunderland
5,Newcastle,Newcastle United,90,1,Newcastle United
6,West Bromwich Albion,West Bromwich Albion,100,1,West Bromwich Albion
7,Norwich City,Norwich City,100,1,Norwich City
8,Queens Park Rangers,Queens Park Rangers,100,1,Queens Park Rangers
9,Swansea City,Swansea City,100,2,Swansea City


In [304]:
table_all_years = pd.read_excel('All_teams_results_leagues_countries_2001_2021_edited.xlsx')
table_all_years = table_all_years.drop(['Unnamed: 0'], axis = 1)

['Vitória Setúbal',
 'Hull City',
 'Köln',
 'Caen',
 'Liverpool',
 'Southampton',
 'Arsenal',
 'Bastia',
 'Málaga',
 'Frosinone',
 'Le Mans',
 'Reggina',
 'West Ham',
 'Levante',
 'Ancona',
 'Parma',
 'West Bromwich Albion',
 'Watford',
 'Boavista',
 'Juventus',
 'Crystal Palace',
 'Real Betis',
 'Villarreal',
 'Leicester',
 'Burnley',
 'Sevilla',
 'Grenoble',
 'Nancy',
 'Marítimo',
 'Olympique Lyonnais',
 'Chievo',
 'Istres',
 'FC Augsburg',
 'Vicenza',
 'Spezia',
 'Campomaiorense',
 'Monaco',
 'Stoke City',
 'Académica',
 'Man. City',
 'Fulham',
 'Paderborn',
 'Deportivo',
 'Estrela Amadora',
 'Alverca',
 'Treviso',
 'Farense',
 'Numancia',
 'Aston Villa',
 'Vitória Guimarães',
 'Greuther Fürth',
 'Cesena',
 'Venezia',
 'Perugia',
 'Hércules',
 '1860 München',
 'Chelsea',
 'Inter',
 'Bradford City',
 'Nacional',
 'Amiens SC',
 'Mallorca',
 'Athletic',
 'Sunderland',
 'Novara',
 'Bologna',
 'Crotone',
 'Os Belenenses',
 'Brighton & Hove Albion',
 'Messina',
 'Alavés',
 'Karlsruher SC'

In [306]:
list_original_teams = list(set(table_all_years[table_all_years['Competition']=='National League']['Home_team'].tolist()))
list_original_teams.sort()
list_original_teams

['1860 München',
 'AFC Bournemouth',
 'Académica',
 'Ajaccio',
 'Alavés',
 'Albacete',
 'Alemannia Aachen',
 'Almería',
 'Alverca',
 'Amiens SC',
 'Ancona',
 'Angers SCO',
 'Arles',
 'Arminia Bielefeld',
 'Arouca',
 'Arsenal',
 'Ascoli',
 'Aston Villa',
 'Atalanta',
 'Athletic',
 'Atlético',
 'Auxerre',
 'B. Dortmund',
 'B. Leverkusen',
 'B. Mönchengladbach',
 'Barcelona',
 'Bastia',
 'Bayern München',
 'Beira Mar SC',
 'Belenenses SAD',
 'Benevento',
 'Benfica',
 'Birmingham City',
 'Blackburn Rovers',
 'Blackpool',
 'Boavista',
 'Bologna',
 'Bolton Wanderers',
 'Bradford City',
 'Braunschweig',
 'Brescia',
 'Brighton & Hove Albion',
 'Burnley',
 'CD Santa Clara',
 'Caen',
 'Cagliari',
 'Campomaiorense',
 'Cardiff City',
 'Carpi',
 'Catania',
 'Celta',
 'Cesena',
 'Charlton Athletic',
 'Chaves',
 'Chelsea',
 'Chievo',
 'Como',
 'Coventry City',
 'Crotone',
 'Crystal Palace',
 'Cádiz',
 'Córdoba',
 'Darmstadt 98',
 'Deportivo',
 'Derby County',
 'Desportivo Aves',
 'Dijon FCO',
 'Eibar

In [307]:
list_fifa_teams = list(set(table_fifa_all_years['Name'].tolist()))
list_fifa_teams.sort()
list_fifa_teams

['1. FC Kaiserslautern',
 '1. FC Köln',
 '1. FC Nuremberg',
 '1. FC Nürnberg',
 '1. FC Union Berlin',
 '1. FSV Mainz 05',
 '1899 Hoffenheim',
 'A.C. Cesena',
 'A.S. Nancy Lorraine',
 'A.S. Nancy-Lorraine',
 'A.S. Saint-Etienne',
 'AC Ajaccio',
 'AC Arles Avignon',
 'AC Milan',
 'ACF Fiorentina',
 'AFC Bournemouth',
 'AJ Auxerre',
 'AS Monaco',
 'AS Monaco FC',
 'AS Monaco Football Club SA',
 'AS Nancy Lorraine',
 'AS Nancy-Lorraine',
 'AS Roma',
 'AS Saint-Etienne',
 'AS Saint-Étienne',
 'Académica Coimbra',
 'Académica de Coimbra',
 'Albacete',
 'Alemannia Aachen',
 'Amiens SC',
 'Amiens SC Football',
 'Angers SCO',
 'Arminia Bielefeld',
 'Arouca',
 'Arsenal',
 'Arsenal FC',
 'Ascoli',
 'Aston Villa',
 'Atalanta',
 'Atalanta BC',
 'Athletic Club',
 'Athletic Club de Bilbao',
 'Athletic de Bilbao',
 'Atletico de Madrid',
 'Atlético Madrid',
 'Atlético de Madrid',
 'Bari',
 'Bayer 04 Leverkusen',
 'Bayer Leverkusen',
 'Bayern Munich',
 'Bayern München',
 'Belenenses',
 'Belém',
 'Beneve

In [308]:
list_original_teams = list(set(table_all_years[table_all_years['Competition']=='National League']['Home_team'].tolist()))
list_fifa_teams = list(set(table_fifa_all_years['Name'].tolist()))

d = {'All_names_original': list_original_teams,
     'All_names_fifa': list_fifa_teams
    }
df1 = pd.DataFrame(data=d)

df1

ValueError: arrays must all be same length

In [66]:
df_fifa_2005 = table_fifa_all_years[table_fifa_all_years['Year'] == 2005]
list_teams_fifa05 = list(df_fifa_2005['Name'])
len(list_teams_fifa05)

116

In [121]:
table_all_years = pd.read_excel('All_teams_results_leagues_countries_2001_2021_edited.xlsx')
table_all_years = table_all_years.drop(['Unnamed: 0'], axis = 1)
table_all_years.head()

,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team
0,2001,International,09 Nov 00,2000-11-09,Europa League,Europa League,Roma,Boavista,1 - 1,1,1,x,1,1,4,4
1,2001,International,26 Oct 00,2000-10-26,Europa League,Europa League,Boavista,Roma,0 - 1,0,1,2,0,3,2,2
2,2001,International,28 Sep 00,2000-09-28,Europa League,Europa League,Boavista,Vorskla Poltava,2 - 1,2,1,1,3,0,4,1
3,2001,International,14 Sep 00,2000-09-14,Europa League,Europa League,Vorskla Poltava,Boavista,1 - 2,1,2,2,0,3,0,3
4,2001,Portugal,27 May 01,2001-05-27,Liga Portuguesa,National League,Porto,Boavista,4 - 0,4,0,1,3,0,4,2


In [69]:
table_all_years.shape

(63257, 15)

In [154]:
df_teams_fifa_05_06 = table_fifa_all_years[(
    table_fifa_all_years['Year'] == 2005) | (table_fifa_all_years['Year'] == 2006
)]

list_teams_fifa_05_06 = list(set(list(df_teams_fifa_05_06['Name'])))
print(len(list_teams_fifa_05_06))
list_teams_fifa_05_06.sort()
print(list_teams_fifa_05_06)

138
['AC Ajaccio', 'AC Milan', 'AJ Auxerre', 'AS Monaco', 'AS Nancy-Lorraine', 'AS Roma', 'AS Saint-Etienne', 'Albacete', 'Arminia Bielefeld', 'Arsenal', 'Ascoli', 'Aston Villa', 'Atalanta BC', 'Athletic de Bilbao', 'Atletico de Madrid', 'Atlético Madrid', 'Bayer Leverkusen', 'Bayern Munich', 'Belenenses', 'Benfica', 'Birmingham City', 'Blackburn Rovers', 'Boavista', 'Bologna', 'Bolton Wanderers', 'Bordeaux', 'Borussia Dortmund', 'Brescia', 'Cagliari', 'Charlton Athletic', 'Chelsea', 'Chievo Verona', 'Coimbra', 'Crystal Palace', 'Cádiz', 'Deport. Alavés', 'Doria', 'ES Troyes AC', 'Eintracht Frankfurt', 'Empoli', 'Estoril Praia', 'Estrela Amadora', 'Everton', 'F.C. Penafiel', 'F.C. Porto', 'FC Barcelona', 'FC Cologne', 'FC Hansa Rostock', 'FC Kaiserslautern', 'FC Metz', 'FC Nantes', 'FC Nürnberg', 'FC Porto', 'FC Schalke 04', 'Fiorentina', 'Firenze', 'Fulham', 'Getafe', 'Gil Vicente', 'Hamburger Sport Verein', 'Hannover 96', 'Hertha BSC Berlin', 'Inter Milan', 'Istres', 'Juventus', 'Laz

In [155]:
df_teams_05_06 = table_all_years[
    ((table_all_years['Year'] == 2005) | (table_all_years['Year'] == 2006)) &
    (table_all_years['Competition'] == 'National League')
]

list_teams_05_06 = list(set(list(df_teams_05_06['Home_team']) + list(df_teams_05_06['Away_team'])))
print(len(list_teams_05_06))
list_teams_05_06.sort()
print(list_teams_05_06)

134
['Académica', 'Ajaccio', 'Alavés', 'Albacete', 'Arminia Bielefeld', 'Arsenal', 'Ascoli', 'Aston Villa', 'Atalanta', 'Athletic', 'Atlético', 'Auxerre', 'B. Dortmund', 'B. Leverkusen', 'B. Mönchengladbach', 'Barcelona', 'Bastia', 'Bayern München', 'Beira Mar SC', 'Benfica', 'Birmingham City', 'Blackburn Rovers', 'Boavista', 'Bologna', 'Bolton Wanderers', 'Brescia', 'Caen', 'Cagliari', 'Celta', 'Charlton Athletic', 'Chelsea', 'Chievo', 'Crystal Palace', 'Cádiz', 'Deportivo', 'Eintracht Frankfurt', 'Empoli', 'Espanyol', 'Estoril', 'Estrela Amadora', 'Everton', 'Fiorentina', 'Fulham', 'Getafe', 'Gil Vicente', 'Girondins Bordeaux', 'Hamburger SV', 'Hannover 96', 'Hansa Rostock', 'Hertha BSC', 'Inter', 'Istres', 'Juventus', 'Kaiserslautern', 'Köln', 'Lazio', 'Le Mans', 'Lecce', 'Lens', 'Levante', 'Lille', 'Liverpool', 'Livorno', 'MSV Duisburg', 'Mainz 05', 'Mallorca', 'Man. City', 'Man. Utd', 'Marítimo', 'Messina', 'Metz', 'Middlesbrough', 'Milan', 'Monaco', 'Moreirense', 'Málaga', 'Nacio

In [160]:
Years = range(2012,2015)

Countries = ['Portugal','Spain','England','Italy','Germany','France']

print('Fifa teams:\n')
for year in Years:
    for country in Countries:
        df_teams_fifa = table_fifa_all_years[((
        table_fifa_all_years['Year'] == int(f'{year}')) &
        (table_fifa_all_years['Country'] == f'{country}'
    ))]
        list_teams_fifa = list(set(list(df_teams_fifa['Name'])))
        print(f'{year},{country},',len(list_teams_fifa))

print('\nResultados Futbol teams:\n')
for year in Years:
    for country in Countries:
        df_teams = table_all_years[(
        (table_all_years['Year'] == int(f'{year}')) &
        (table_all_years['Country'] == f'{country}') &
        (table_all_years['Competition'] == 'National League') 
    )]
        list_teams = list(set(list(df_teams['Home_team'])))
        print(f'{year},{country},',len(list_teams))

Fifa teams:

2012,Portugal, 16
2012,Spain, 20
2012,England, 19
2012,Italy, 20
2012,Germany, 18
2012,France, 20
2013,Portugal, 16
2013,Spain, 20
2013,England, 19
2013,Italy, 20
2013,Germany, 18
2013,France, 20
2014,Portugal, 16
2014,Spain, 18
2014,England, 20
2014,Italy, 20
2014,Germany, 18
2014,France, 20

Resultados Futbol teams:

2012,Portugal, 16
2012,Spain, 20
2012,England, 20
2012,Italy, 20
2012,Germany, 18
2012,France, 20
2013,Portugal, 16
2013,Spain, 20
2013,England, 20
2013,Italy, 20
2013,Germany, 18
2013,France, 20
2014,Portugal, 16
2014,Spain, 20
2014,England, 20
2014,Italy, 20
2014,Germany, 18
2014,France, 20


In [126]:
table_fifa_all_years

,Year,Country,Name,ATT,MID,DEF,OVR
0,2005,Portugal,FC Porto,85.0,82.0,77.0,81.0
1,2005,Portugal,Benfica,80.0,74.0,74.0,75.0
2,2005,Portugal,Sporting Lisbon,63.0,70.0,65.0,68.0
3,2005,Portugal,Belenenses,67.0,62.0,62.0,64.0
4,2005,Portugal,Nacional,69.0,59.0,61.0,63.0
...,...,...,...,...,...,...,...
1947,2021,France,FC Nantes,71.0,74.0,74.0,73.0
1948,2021,France,Nîmes Olympique,71.0,73.0,73.0,73.0
1949,2021,France,FC Metz,70.0,70.0,73.0,72.0
1950,2021,France,Dijon FCO,73.0,71.0,70.0,71.0


In [164]:
table_fifa_all_years[
    (table_fifa_all_years['Country']=='Spain') &
    (table_fifa_all_years['Year']==2014)
                    ]

,Year,Country,Name,ATT,MID,DEF,OVR
1044,2014,Spain,Real Madrid Club de Fútbol,88.0,83.0,82.0,84.0
1045,2014,Spain,FC Barcelona,84.0,86.0,84.0,84.0
1046,2014,Spain,Club Atlético de Madrid,81.0,80.0,81.0,81.0
1047,2014,Spain,Real Sociedad de Fútbol,80.0,76.0,76.0,77.0
1048,2014,Spain,Valencia Club de Fútbol,76.0,77.0,76.0,77.0
1049,2014,Spain,Sevilla Fútbol Club,78.0,75.0,75.0,76.0
1050,2014,Spain,Athletic Club de Bilbao,77.0,78.0,75.0,76.0
1051,2014,Spain,Málaga Club de Fútbol,75.0,73.0,76.0,76.0
1052,2014,Spain,Real Betis Balompié,74.0,72.0,73.0,74.0
1053,2014,Spain,Getafe Club de Fútbol,73.0,73.0,74.0,74.0
